<a href="https://colab.research.google.com/github/Samir-atra/Expenses_tracker/blob/main/project/utils/image_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import pathlib
import IPython
import sys
from sklearn.model_selection import train_test_split
import cv2
import numpy as np

In [26]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [50]:
data_dir = pathlib.Path('/content/drive/MyDrive/expenses_tracker_classifier/Tobacco3482-jpg/')

def load_data(dir):
    img_data_array = []
    class_name = []
    class_num = 0

    for dir1 in os.listdir(dir):
        # print("this is dir1", dir1)
        for filey in os.listdir(os.path.join(dir, dir1)):
            image_path = os.path.join(dir, dir1, filey)
            image = cv2.imread(image_path, cv2.COLOR_BGR2RGB)
            if image is None:
                continue
            image = cv2.resize(image, (300, 300))
            image = np.array(image)
            image = image.astype("float32")
            # image /= 255
            if len(image.shape) < 3:
                continue
            else:
                img_data_array.append(image)
                class_name.append(class_num)
        class_num +=1
        # print(class_num, dir1)
    return img_data_array, class_name, class_num




# Split data into training and testing sets
img_data_array, class_name, class_num = load_data(data_dir)
# labels = tf.keras.utils.to_categorical(class_name)
# print(labels)
x_train, x_test, y_train, y_test = train_test_split(np.array(img_data_array), np.array(class_name), test_size=0.3)

# data_path_test = pathlib.Path('/content/drive/MyDrive/archiveX3/Testing/')
# AUTOTUNE=tf.data.AUTOTUNE

# dataset_path = tf.keras.utils.image_dataset_from_directory(        # Training dataset
#     data_path,
#     labels= 'inferred',
#     validation_split=0.2,
#     subset='training',
#     seed= 1,
#     batch_size=5,
#     image_size=(299, 299),
#     color_mode="rgb",
#     shuffle=True)#.cache().prefetch(buffer_size=AUTOTUNE)

# dataset_path_val = tf.keras.utils.image_dataset_from_directory(      #Validation dataset
#     data_path,
#     labels= 'inferred',
#     validation_split=0.2,
#     subset='validation',
#     seed= 2,
#     batch_size=5,
#     image_size=(299, 299),
#     color_mode="rgb",
#     shuffle=True)#.cache().prefetch(buffer_size=AUTOTUNE)

796
[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,

In [39]:
base_model = tf.keras.applications.inception_v3.InceptionV3(
    input_shape = (300, 300, 3),
    include_top = False,
    weights = "imagenet"
)

In [40]:
saving_path = pathlib.Path('/content/drive/MyDrive/expenses_tracker_classifier/expenses_tracker_classifier95%test.h5')

base_model.save(saving_path)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [41]:
model_path = pathlib.Path('/content/drive/MyDrive/expenses_tracker_classifier/expenses_tracker_classifier95%test.h5')

base_model = tf.keras.models.load_model(model_path)                 #Loading base_model

base_model.trainable = False                                        # Setting the model as non-trainable

rescaling = tf.keras.Sequential([
  tf.keras.layers.Rescaling(scale=1 / 127.5, offset=-1)             # Rescaling to (1, -1) range required for inceptionV3 model
])
augmentation = tf.keras.Sequential([                                # Applying augmentations o the images
   tf.keras.layers.RandomFlip("horizontal"),
   tf.keras.layers.RandomRotation(0.1)
])


inputs = tf.keras.Input(shape=(300, 300, 3))
x = augmentation(inputs)
x = rescaling(x)
x = base_model(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
outputs = tf.keras.layers.Dense(12, activation='softmax')(x)                   # "softmax" in the final layer for decision making
model = tf.keras.Model(inputs, outputs)


model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0005),        # compiling with low learning rate
    loss=tf.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy'],
    run_eagerly = True)

model.fit(                                                               # fitting the whole model for non-trainable base
    x_train,
    y_train,
    epochs=30,)

model.evaluate(
    x=x_test,
    y=y_test,)


Epoch 1/30
18/18 [==============================] - 11s 504ms/step - loss: 1.3801 - accuracy: 0.4542
Epoch 2/30
18/18 [==============================] - 10s 535ms/step - loss: 0.2591 - accuracy: 0.9515
Epoch 3/30
18/18 [==============================] - 10s 545ms/step - loss: 0.1034 - accuracy: 0.9946
Epoch 4/30
18/18 [==============================] - 9s 514ms/step - loss: 0.0645 - accuracy: 0.9982
Epoch 5/30
18/18 [==============================] - 10s 535ms/step - loss: 0.0492 - accuracy: 0.9982
Epoch 6/30
18/18 [==============================] - 10s 544ms/step - loss: 0.0360 - accuracy: 1.0000
Epoch 7/30
18/18 [==============================] - 9s 520ms/step - loss: 0.0333 - accuracy: 1.0000
Epoch 8/30
18/18 [==============================] - 9s 514ms/step - loss: 0.0280 - accuracy: 1.0000
Epoch 9/30
18/18 [==============================] - 10s 551ms/step - loss: 0.0225 - accuracy: 1.0000
Epoch 10/30
18/18 [==============================] - 10s 542ms/step - loss: 0.0219 - accuracy:

[0.027193671092391014, 0.9958158731460571]

In [48]:
loading_path = pathlib.Path('/content/drive/MyDrive/expenses_tracker_classifier/expenses_tracker_classifier95%test.h5')
loaded_model = tf.keras.models.load_model(loading_path, compile=True)
pred_img_path = pathlib.Path('/content/drive/MyDrive/expenses_tracker_classifier/pred_imgs/')
# images, labels = load_data(pred_img_path)
# print("this is iamges", len(labels), labels)
# print("this is labels", labels)

loaded_model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0005),        # compiling with low learning rate
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits= True),
    metrics=['accuracy'],
    run_eagerly = False)

dataset_pathy = tf.keras.utils.image_dataset_from_directory(
    pred_img_path,
    labels= 'inferred',
    seed= 1,
    batch_size=1,
    image_size=(300, 300),
    color_mode="rgb",
    shuffle=False)


# # for image in images:
#   #  for i in range(1):
#       #  ax = plt.subplot(1, 1, i+1)
#    plt.imshow(images[0].astype("uint8"))
#       #  plt.title(class_names[labels[i]])
#       #  plt.axis("off")
#    plt.show()


sample_list = []                                                        # creating a list of sample(s)
for sample, label in dataset_pathy:
    sample_list.append(sample)
    continue

sample_list = np.array(sample_list)
for sample in sample_list:
    print(sample.shape)
    # images[0] = tf.expand_dims(images[0], axis=0)
    predictions = loaded_model.predict(sample)
    pred = np.argmax(predictions, axis=1)
    print(predictions.shape)
    print(pred.shape)
    if pred[0] == 0:
      print("beedoo")

Found 1 files belonging to 1 classes.
(1, 300, 300, 3)
1/1 [==============================] - 1s 1s/step
(1, 8, 8, 2048)
(1, 8, 2048)


ValueError: ignored